# 第7节 蒙卡模拟计算路径依赖型期权价格
* [7.1 简介](#7_1)
* [7.2 蒙卡模拟计算回望/亚式期权算法](#7_2)
* [7.3 算法 Python 代码实现](#7_3)
* [7.4 计算示例](#7_4)
* [7.5 相关说明](#7_5)
    * [7.5.1 由均匀分布产生正态分布随机数](#7_5_1)
    * [7.5.2 蒙卡模拟结果与解析解比较](#7_5_2)
* [7.6 参考资料](#7_6)

## <a name="7_1"><a/> 7.1 简介
&nbsp; &nbsp; &nbsp; &nbsp; 
蒙特卡罗模拟计算期权价格是将股票价格变化过程离散化，通过随机数产生器对每一小段股价变化进行抽样，累计得到一次股价变化过程抽样结果，然后计算出对应的期权价格。进行多次股价变化过程抽样计算出期权价格的平均价并贴现，即为蒙卡模拟得出的期权价格。  
&nbsp; &nbsp; &nbsp; &nbsp; 
由于抽样股价过程可以得到完整的股价变化路径，所以可以直接由抽样结果得到路径依赖型期权的价格。这是蒙特卡罗模拟计算期权价格相对于树形计算期权价格的一个优势。

## <a name="7_2"><a/> 7.2 蒙卡模拟计算回望/亚式期权价格算法  
&nbsp; &nbsp; &nbsp; &nbsp; 
以浮动回望看涨期权和亚式平均价格看涨期权为例。浮动回望看涨期权执行时的价格为$\max(S_T-S_{min}, \, 0)$，$S_T$为执行时刻股票价格，$S_{min}$为历史股价极小值；平均价格看涨期权执行时价格为$\max(S_{ave}-K,\,0)$，$K$为约定执行价格，$S_{ave}$为历史平均股价。  
1. 根据给定参数，确定股价变化离散化后过程，这里我们使用指数形式。
    $$S(t+\Delta t)=S(t)\,e^{(r-\frac{1}{2}\sigma^2)\Delta t+\varepsilon \sigma\sqrt{\Delta t}},\;\; \varepsilon \sim \mathcal N(0, 1), \;\; \Delta t =\frac{T}{N} .$$  
1. 由$S(t=0)=S_0$开始，依照股价离散化变化过程进行抽样，共$N$次。得到一条股价变化的抽样路径。  
1. 对于浮动回望看涨：由抽样股价变化路径计算出$\max(S_T-S_{min},\,0)$，乘以贴现因子$e^{-rT}$，得到浮动看涨期权的一次抽样价格；对于平均价格看涨：由抽样股价变化路径计算出$\max(S_{ave}-K,\, 0)$，乘以贴现因子$e^{-rT}$，得到平均价格期权的一次抽样价格。  
1. 重复步骤2、3共$M$次，对得到的期权价格取平均值，即为蒙卡模拟计算出的目标期权价格。

## <a name="7_3"><a/> 7.3 算法Python代码实现  

In [17]:
import numpy as np
import math

E = math.e

def sample_S_path(r, sigma, S_0, T, N):
    S = S_0
    path = [S]
    dt = T/N
    for i in range(N):
        S *= E**((r-0.5*sigma*sigma)*dt+sigma*np.random.normal()*dt**0.5)
        path.append(S)
    return path

def MC_float_lookback_call(r, sigma, S_0, T, N, M):
    call_price = 0
    for i in range(M):
        path = sample_S_path(r, sigma, S_0, T, N)
        call_price += 1.0/M*max(0, path[-1]-min(path))
    return call_price*E**(-r*T)

def MC_average_price_call(r, sigma, S_0, K, T, N, M):
    call_price = 0
    for i in range(M):
        path = sample_S_path(r, sigma, S_0, T, N)
        call_price += 1.0/M*max(0, np.average(path)-K)
    return call_price*E**(-r*T)

## <a name="7_4"><a/> 7.4 计算示例  
&nbsp; &nbsp; &nbsp; &nbsp; 
考虑当$r=0.1, \; \sigma = 0.4, \; S_0 = 50, \; T=1$，离散化步数$N=150$，蒙卡抽样股价变化路径数$M=40000$。且对于亚式平均价格看涨期权，执行价格$K=50$。计算浮动回望看涨和平均价格看涨期权价格结果分别如下：

In [40]:
float_lookback_call = MC_float_lookback_call(0.1, 0.4, 50, 1, 150, 40000)
average_price_call = MC_average_price_call(0.1, 0.4, 50, 50, 1, 150, 40000)
print("欧式浮动回望看涨期权价格：{0:.5f}".format(float_lookback_call))
print("亚式平均价格看涨期权价格:{0:.5f}".format(average_price_call))

欧式浮动回望看涨期权价格：15.42357
亚式平均价格看涨期权价格:5.59780


## <a name="7_5"><a/> 7.5 相关说明
### <a name="7_5_1"><a/> 7.5.1 由均匀分布产生正态分布随机数  
&nbsp; &nbsp; &nbsp; &nbsp; 
虽然一般编程语言常用库中都有正态分布随机数产生器。但有些情况下我们可能希望使用比较高质量的均匀分布随机数产生器来产生正态分布样本。一般而言，使用均匀分布抽样其它分布，需要使用目标分布的累积概率函数的逆函数，或者需要一个辅助累积概率函数加上一定概率接受抽样。这里针对标准正态分布，我们介绍一下Box-Muller方法，通过抽样两个均匀分布样本来产生一个标准正态分布样本。  
&nbsp; &nbsp; &nbsp; &nbsp; 
考虑$x, y$坐标上相互独立的二维标准正态分布，其概率密度函数为$f(x,y) = \frac{1}{2\pi}e^{-\frac{x^2+y^2}{2}}$。在极坐标系中，$f(r,\phi) = \frac{1}{2\pi}e^{-\frac{1}{2}r^2}$，且边缘概率函数
    $$F(r) = \int_0^r\int_0^{2\pi}f(s,\phi)\,s\,d\phi\, ds = \int_0^r e^{-\frac{1}{2}s^2}ds=1-e^{-\frac{1}{2}r^2}.$$
其逆函数为 $F^{-1}(x) = \sqrt{-2\ln{(1-x)}}$。由于二维标准正态分布抽样结果的$x$方向投影为一维标准正态分布抽样。在极坐标系下我们先对径向的$r$抽样，再对角向的$\phi$抽样，然后将结果投影到$x$坐标轴。由于$r \sim \sqrt{-2\ln{(1-t)}}\sim \sqrt{-2\ln{t}}$，$t$为$[0,1]$之间均匀分布的随机数。 所以$x$坐标轴上的一个标准正态分布抽样结果可以表示为$\sqrt{-2\ln{t_1}}\cdot \cos{(2\pi t_2)}$，其中$t_1,\,t_2$均为$[0,1]$之间均匀分布随机数抽样。  

### <a name="7_5_2"><a/> 7.5.2 蒙卡模拟结果与解析解比较  
&nbsp; &nbsp; &nbsp; &nbsp; 
参考《期权、期货及其他衍生产品》第26章，我们知道浮动回望看涨期权价格为：
    $$ c_{fl} = S_0N(a_1)-S_0\frac{\sigma^2}{2r}N(-a_1)-S_0e^{-rT}(1-\frac{\sigma^2}{2r})N(a_2).$$
    $$a_1 = (r+\frac{\sigma^2}{2})\frac{\sqrt{T}}{\sigma} ,\;\; a_2 = (r-\frac{\sigma^2}{2})\frac{\sqrt{T}}{\sigma} .$$
代入相关参数，得$c_{fl} = 15.92343$。  
&nbsp; &nbsp; &nbsp; &nbsp; 
亚式平均价格看涨期权价格为：
$$d_1 = \frac{\ln{\frac{M_1}{K}}+\frac{\sigma_0^2}{2}T}{\sigma_0\sqrt{T}},\;\;d_2 = d_1-\sigma_0\sqrt{T},\;\; M_1 = S_0\frac{e^{rT}-1}{rT},\;\;\; \sigma_0^2 = \frac{1}{T}\ln{\frac{M_2}{M_1^2}} .$$
$$M_2=\frac{2e^{(2r+\sigma^2)T}S_0^2}{(r+\sigma^2)(2r+\sigma^2)T^2}+\frac{2S_0^2}{rT^2}\left(\frac{1}{2r+\sigma^2}-\frac{e^{rT}}{r+\sigma^2}\right). $$
    $$ c_{ave} = e^{-rT}(M_1N(d_1)-KN(d_2)) .\qquad\qquad\qquad\qquad\qquad\qquad\qquad\quad$$
代入相关参数，得$c_{ave} = 5.61679$。  
&nbsp; &nbsp; &nbsp; &nbsp; 
当我们取较大的$M,N$，比如$M=200,\;N=640000$时，蒙卡模拟计算出的$c_{fl}^\ast = 15.35872,\; c_{ave}^\ast = 5.55239$，和解析解直接计算出的结果很接近。

## <a name="7_6"><a/> 7.6 参考资料
1. 《期权、期货及其他衍生产品》，John C. Hull 著，王勇、索吾林译。

In [37]:
from scipy.stats import norm

r = 0.1
sigma = 0.4 
S_0 = 50 
K = 50 
T = 1

a_1 = (r+0.5*sigma*sigma)*T**0.5/sigma
a_2 = (r-0.5*sigma*sigma)*T**0.5/sigma
c_fl = S_0*norm.cdf(a_1)-S_0*sigma*sigma/2/r*norm.cdf(-a_1)-S_0*E**(-r*T)*(1-sigma*sigma/2/r)*norm.cdf(a_2)

M_1 = S_0*(E**(r*T)-1)/r/T
M_2 = 2*E**((2*r+sigma*sigma)*T)*S_0*S_0/(r+sigma*sigma)/(2*r+sigma*sigma)/T/T
M_2 += 2*S_0*S_0/r/T/T*(1/(2*r+sigma*sigma)-E**(r*T)/(r+sigma*sigma))
sigma_0 = (1/T*math.log(M_2/M_1/M_1))**0.5

d_1 = (math.log(M_1/K)+(0.5*sigma_0*sigma_0)*T)/sigma_0/T**0.5
d_2 = d_1 - sigma_0*T**0.5
c_ave = E**(-r*T)*(M_1*norm.cdf(d_1)-K*norm.cdf(d_2))

c_fl, c_ave

(15.923429307972278, 5.616791502292813)